# MATRIX CALCULATIONS with NUMPY
####    from Harel Wallach-15Oct2019- modified by Iaakov1-15Oct2019
####    modified Harel Wallach-17Oct2019

In [2]:
import numpy as np
from numpy import linalg as LA
import math
import pandas as pd
import matplotlib as mpl
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.render()
    display_html(html_str.replace('table','table style="display:inline;padding:15px;"'),raw=True)


Paste matrices as tab-separated

In [3]:
provider_str = """
1	0	0	0	0	0	0
0	1	0	0	0	0	0
0	0	1	0	0	0	0
0	0	0	1	0	0	0
0	0	0	0	1	0	0
0	0	0	0	0	1	0
0	0	0	0	0	0	1
"""

consumer_str = """
0	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	1	1	1	1
1	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	0	0	0	0
"""

Read the matrices into Numpy

In [4]:
#provider_str = provider_str.replace('\t','\t0 ')
mat = np.fromstring(provider_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
provider_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

#consumer_str = consumer_str.replace('\t','\t0 ')
mat = np.fromstring(consumer_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
consumer_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

Combine two matrices to the wheighted Modularity Matrix

In [5]:
provider_wheight = 2
consumer_wheight = 1
union_mat = provider_mat*provider_wheight + consumer_mat*consumer_wheight

Let's view:

In [6]:
header = ['S{}'.format(i) for i in range(1, dim+1)]
indx = ['F{}'.format(i) for i in range(1, dim+1)]
provider_df = pd.DataFrame(provider_mat, columns= header, index = indx)
consumer_df = pd.DataFrame(consumer_mat, columns= header, index = indx)
union_df = pd.DataFrame(union_mat, columns= header, index = indx)

p=provider_df.style.background_gradient(cmap='Wistia').set_caption("PROVIDERS")
c=consumer_df.style.background_gradient(cmap='Wistia').set_caption("CONSUMERS")
u=union_df.style.background_gradient(cmap='gist_gray').set_caption("UNION")
display_side_by_side(p,c,u)

,S1,S2,S3,S4,S5,S6,S7
F1,1,0,0,0,0,0,0
F2,0,1,0,0,0,0,0
F3,0,0,1,0,0,0,0
F4,0,0,0,1,0,0,0
F5,0,0,0,0,1,0,0
F6,0,0,0,0,0,1,0
F7,0,0,0,0,0,0,1
,S1,S2,S3,S4,S5,S6,S7
F1,0,0,0,0,0,0,0
F2,1,0,0,0,0,0,0


In [7]:
## This is used for further iterations
#union_str = """
#2	1	1	1	1
#0	2	0	0	0
#0	0	2	0	0
#0	0	0	2	0
#0	0	0	0	2
#"""
#mat = np.fromstring(union_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
#dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
#union_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

In [8]:
row_sum = np.sum(union_mat,axis = 1, dtype=int) #functionals sum
col_sum = np.sum(union_mat,axis = 0, dtype=int) #structors sum
deg_mat = np.diag(np.concatenate((row_sum,col_sum)))

In [9]:
adj_mat = np.block([[np.zeros((dim,dim)), union_mat], [union_mat.T,np.zeros((dim,dim))]])
laplacian = np.matrix(deg_mat-adj_mat, dtype=int)

In [10]:
deg_df = pd.DataFrame(deg_mat, columns= indx+header, index = indx+header)
adj_df = pd.DataFrame(adj_mat, columns= indx+header, index = indx+header)
laplacian_df = pd.DataFrame(laplacian, columns= indx+header, index = indx+header)

de=deg_df.style.background_gradient(cmap='Wistia').set_caption("DEGREES")
ad=adj_df.style.background_gradient(cmap='Wistia').set_caption("ADJACENCY")
lap=laplacian_df.style.background_gradient(cmap='Wistia').set_caption("LAPLACIAN (D-A)")
display_side_by_side(de,ad)
display(lap)

,F1,F2,F3,F4,F5,F6,F7,S1,S2,S3,S4,S5,S6,S7
F1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
F2,0,3,0,0,0,0,0,0,0,0,0,0,0,0
F3,0,0,7,0,0,0,0,0,0,0,0,0,0,0
F4,0,0,0,3,0,0,0,0,0,0,0,0,0,0
F5,0,0,0,0,3,0,0,0,0,0,0,0,0,0
F6,0,0,0,0,0,3,0,0,0,0,0,0,0,0
F7,0,0,0,0,0,0,3,0,0,0,0,0,0,0
S1,0,0,0,0,0,0,0,8,0,0,0,0,0,0
S2,0,0,0,0,0,0,0,0,2,0,0,0,0,0
S3,0,0,0,0,0,0,0,0,0,2,0,0,0,0


,F1,F2,F3,F4,F5,F6,F7,S1,S2,S3,S4,S5,S6,S7
F1,2,0,0,0,0,0,0,-2,0,0,0,0,0,0
F2,0,3,0,0,0,0,0,-1,-2,0,0,0,0,0
F3,0,0,7,0,0,0,0,-1,0,-2,-1,-1,-1,-1
F4,0,0,0,3,0,0,0,-1,0,0,-2,0,0,0
F5,0,0,0,0,3,0,0,-1,0,0,0,-2,0,0
F6,0,0,0,0,0,3,0,-1,0,0,0,0,-2,0
F7,0,0,0,0,0,0,3,-1,0,0,0,0,0,-2
S1,-2,-1,-1,-1,-1,-1,-1,8,0,0,0,0,0,0
S2,0,-2,0,0,0,0,0,0,2,0,0,0,0,0
S3,0,0,-2,0,0,0,0,0,0,2,0,0,0,0


In [11]:
# Iaakov: added COMPUTATIONS
# Matrix RANK
sqmat_rank = np.linalg.matrix_rank(laplacian)
# Matrix TRACE
sqmat_trace = np.trace(laplacian)
# the EIGENV computation
v, w = LA.eig(laplacian)

In [12]:
# Iaakov: changed PRINTING precision from 10 to 2
np.set_printoptions(linewidth=200, precision=2,formatter={'float_kind':'{:5.3f}'.format},)

# Iaakov: more pretty PRINTING
#print('\n==== MATRIX CALCULATIONS with NUMPY ====')
#print('\nINPUT MATRIX')
#print(laplacian)
print("\n--->Matrix RANK: ", sqmat_rank)
print("-->Matrix TRACE:", sqmat_trace)


--->Matrix RANK:  13
-->Matrix TRACE: 48


### find the indecies of the fisrt k' minimal elements of the array
res_df.columns[np.argpartition(v,3)[1]] will give the name of the column with second minimal value

In [13]:
A = np.array([1, 7, 9, 2, 0.1, 17, 17, 1.5])
k = 4 
#find the indecies of the fisrt k' minimal elements of the array
idx = np.argpartition(A, k)
print(idx)
print(A[idx[1]])

[4 0 7 3 1 2 6 5]
1.0


In [18]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'repeating-linear-gradient(to right,grey,grey 2px,white 2px,grey 2px)' if val < 0 else 'rgb(88, 191, 238)'
    return 'background-color: %s' % color
vheader = ['V{}: '.format(i) + str(x) for i,x in zip(range(1, v.size+1),np.around(v,3))]
res_df = pd.DataFrame(np.around(w,5), columns= vheader, index = header+indx)
pd.options.display.max_colwidth = 130
res=res_df.style.\
    applymap (color_negative_red, subset=res_df.columns[np.argpartition(v,3)[1]]).\
    set_caption("EIGENVALUES & related EIGENVECTORS")
display(res)      

,V1: 10.009,V2: 7.775,V3: 4.226,V4: -0.0,V5: 1.643,V6: 1.348,V7: 0.438,V8: 4.562,V9: 5.0,V10: 5.0,V11: 1.0,V12: 1.0,V13: 1.0,V14: 5.0
S1,0.19074,0.16994,0.20106,-0.26726,0.66773,-0.61402,0,-0,-0,-0,0,-0,0,0
S2,0.11734,0.1202,-0.3914,-0.26726,-0.01214,0.04468,-0.56126,0.65575,-0,-0,0,-0,-0,0
S3,0.47631,-0.74981,0.1889,-0.26726,0.09752,0.22467,0.13613,0.15904,-0,-0,-0,-0,-0,0
S4,0.13975,0.04488,-0.26103,-0.26726,-0.16552,-0.09331,0.10628,-0.2037,-0.59348,-0.15541,0.3857,0.51852,0.00743,0.36578
S5,0.13975,0.04488,-0.26103,-0.26726,-0.16552,-0.09331,0.10628,-0.2037,0.24818,-0.34577,0.08205,-0.45283,0.56451,-0.45784
S6,0.13975,0.04488,-0.26103,-0.26726,-0.16552,-0.09331,0.10628,-0.2037,0.0574,0.59022,-0.57674,0.07653,-0.19185,-0.22997
S7,0.13975,0.04488,-0.26103,-0.26726,-0.16552,-0.09331,0.10628,-0.2037,0.2879,-0.08904,0.10899,-0.14223,-0.38009,0.32203
F1,-0.76377,-0.49075,-0.22374,-0.26726,0.11935,-0.20022,0,0,0,0,0,-0,0,-0
F2,-0.0293,-0.04162,0.35172,-0.26726,-0.06792,0.13702,-0.71885,-0.51199,0,0,0,0,-0,-0
F3,-0.11895,0.25965,-0.16975,-0.26726,0.5456,0.68898,0.17435,-0.12418,0,-0,-0,-0,-0,-0
